# 다시 만난 쿠키 문제

- 이번에는 쿠키 문제를 PMF(확률질량함수; Probability Massive Function)를 사용해 다시 풀어보겠다. 
- 확률질량함수를 사용하기 위해 다음과 같이 라이브러리를 호출하자.

```python
from empiricaldist import Pmf
```

쿠키그릇 2개가 있고, 하나는 바닐라 30개, 초코 10개가 들어있다. 다른 하나에는 바닐라 20개, 초코 20개가 들어있다. 
어떤 그릇인지 확인하지 않고 임의로 그릇을 하나 골라서 거기에서 쿠키를 하나 집었다고 하자. 그 때 그 쿠키가 바닐라 쿠키였다면, 이 바닐라 쿠키가 1번 그릇에서 나왔을 확률은 얼마일까?

In [5]:
from empiricaldist import Pmf

prior = Pmf.from_seq(['Bowl 1', 'Bowl 2'])
likelihood_vanilla = [0.75, 0.5]
posterior = prior * likelihood_vanilla
print(posterior)

Bowl 1    0.375
Bowl 2    0.250
Name: proportion, dtype: float64


In [6]:
posterior.normalize()

0.625

In [7]:
posterior

,probs
Bowl 1,0.6
Bowl 2,0.4


저자에 따르면, Pmf 객체는 베이즈테이블을 사용할 때보다 더 많은 데이터에 대해 갱신이 쉽다고 한다. 
예를 들어, 꺼냈던 쿠키를 다시 넣고 같은 그릇에서 쿠키를 꺼냈을 때 이 쿠키가 바닐라라면 다음과 같이 갱신을 할 수 있다.

In [8]:
posterior *= likelihood_vanilla 
posterior.normalize()
posterior

,probs
Bowl 1,0.692308
Bowl 2,0.307692


이제 그릇 1의 사후확률은 거의 70%가 되었다. 하지만 다시 이와 동일한 행동을 했고 이번엔 초콜릿 쿠키가 나왔다.
새로운 데이터에 대한 가능도는 다음과 같다.

In [9]:
likelihood_chocolate = [0.25,0.5]
posterior *= likelihood_chocolate
posterior.normalize()
posterior

,probs
Bowl 1,0.529412
Bowl 2,0.470588


그릇 1의 사후확률은 약 53%이다. 두 개의 바닐라쿠키와 한 개의 초콜릿쿠키를 꺼낸 후 사후확률은 50:50에 보다 가깝게 되었다.

# 101개의 쿠키 그릇

이번에는 101개의 쿠키그릇 문제를 풀어보자.
- 그릇 0에는 바닐라쿠키가 0% 있다.
- 그릇 1에는 바닐라쿠키가 1% 있다.
- 그릇 2에는 바닐라쿠키가 2% 있다.
- ...그릇 99에는 바닐라쿠키가 99%, 그릇 100에는 바닐라쿠키만 있다.
- 여기서 임의의 그릇을 골라서 임의의 쿠키를 꺼냈는데, 이 쿠키가 바닐라쿠키였다. 이 때 각 값 x에 대해 그릇 x에서 쿠키가 나왔을 확률은 얼마일까?

In [10]:
import numpy as np

hypos = np.arange(101)
print(hypos)
prior = Pmf(1,hypos) # Pmf(prior, sequence)
prior.normalize()

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100]


101

In [11]:
prior.head()

,probs
0,0.009901
1,0.009901
2,0.009901


In [12]:
# 데이터의 가능도 = 각 그릇에 있는 바닐라 쿠키의 비율
likelihood_vanilla = hypos/100
likelihood_vanilla[:5]

array([0.  , 0.01, 0.02, 0.03, 0.04])

In [13]:
posterior1 = prior * likelihood_vanilla
posterior1.normalize()
posterior1.head()

,probs
0,0.000000
1,0.000198
2,0.000396


In [24]:
# 잡은 쿠키를 돌려놓고, 동일한 그릇에서 한번 더 뽑았을 때
posterior2 = posterior1 * likelihood_vanilla
posterior2.normalize()

likelihood_chocolate = 1 - hypos/100
posterior3 = posterior2 * likelihood_chocolate
posterior3.normalize()

0.2462686567164179

In [25]:
posterior3.idxmax()

67

# 주사위 문제

앞서 베이즈테이블로 풀어보았던 주사위 문제를 Pmf 를 사용해서 풀어보자.
- 6면체, 8면체, 12면체 주사위가 든 상자가 있다고 할 때 이 중 주사위 하나를 임의로 집어 굴렸더니 1이 나왔다. 6면체 주사위를 굴렸을 확률은 얼마일까?

In [26]:
hypos = [6,8,12]
prior = Pmf(1/3, hypos)
prior

,probs
6,0.333333
8,0.333333
12,0.333333


In [27]:
# Pmf 객체의 두 가지 속성, qs = 분포에 포함되는 원소; ps = 이에 해당하는 확률
print(prior.qs)
print(prior.ps)

[ 6  8 12]
[0.33333333 0.33333333 0.33333333]


In [29]:
likelihood1 = 1/6, 1/8, 1/12
posterior = prior * likelihood1
posterior.normalize()
posterior

,probs
6,0.444444
8,0.333333
12,0.222222


In [30]:
likelihood1

(0.16666666666666666, 0.125, 0.08333333333333333)

이 주사위에서 이번에는 7의 숫자가 나왔다 하자. 그렇다면 이 때의 가능도는 다음과 같다.

In [31]:
likelihood2 = 0, 1/8, 1/12
posterior *= likelihood2
posterior.normalize()
posterior

,probs
6,0.000000
8,0.692308
12,0.307692


In [41]:
def update_dice(pmf:tuple,data):
    """새로운 데이터를 기반으로 pmf를 갱신함
    - pmf: (가능한 주사위, 각각의 확률)
    - data: 주사위를 굴렸을 떄의 결과값
    """
    hypos = pmf.qs
    likelihood = 1/ hypos
    impossible = (data > hypos)
    likelihood[impossible] = 0
    print(pmf)
    pmf *= likelihood
    print(pmf)
    pmf.normalize()

In [33]:
pmf = prior.copy()
pmf

,probs
6,0.333333
8,0.333333
12,0.333333


In [34]:
update_dice(pmf,1)
update_dice(pmf,7)
pmf

,probs
6,0.000000
8,0.692308
12,0.307692


# 문제 3-1

- 육면체, 팔면체, 십이면체 주사위가 든 상자가 있다. 여기서 임의로 주사위 하나를 꺼내서 4번 굴렸을 때, 1,3,5,7이 나왔다. 내가 고른 주사위가 팔면체일 확률은 얼마인가?
- update_dice 함수를 이용하거나 직접 갱신해도 된다.

In [37]:
dice2 = prior.copy()
print(dice2)

6     0.333333
8     0.333333
12    0.333333
dtype: float64


In [38]:
for i in [1,3,5,7]:
    update_dice(dice2, i)
    print(dice2)

6     0.444444
8     0.333333
12    0.222222
dtype: float64
6     0.551724
8     0.310345
12    0.137931
dtype: float64
6     0.646465
8     0.272727
12    0.080808
dtype: float64
6     0.000000
8     0.835052
12    0.164948
dtype: float64


# 문제 3-2

- 문제 3-1에서 상자에는 주사위가 종류별로 하나씩 있어 각 주사위의 사전확률이 동일했다. 하지만 이번에는 상자에 사면체 주사위 하나, 육면체 주사위 두 개, 팔면체 주사위 3개, 십이면체 주사위 4개, 이십면체 주사위 5개가 있다. 
- 이 중 주사위 하나를 골라서 던졌더니 7이 나왔다. 이 때 내가 팔면체 주사위를 골랐을 확률은 얼마일까?
- Hint: Pmf()에 두 개의 매개변수를 넣어 사전확률분포를 만들 수 있다.

In [42]:
ps = [1,2,3,4,5]
qs = [4,6,8,12,20]
pmf = Pmf(ps,qs)
print(pmf)
update_dice(pmf,7)
print(pmf)

4     1
6     2
8     3
12    4
20    5
dtype: int64
4     1
6     2
8     3
12    4
20    5
dtype: int64
4     0.000000
6     0.000000
8     0.375000
12    0.333333
20    0.250000
dtype: float64
4     0.000000
6     0.000000
8     0.391304
12    0.347826
20    0.260870
dtype: float64


# 문제 3-3

- 두 개의 양말 서랍이 있다. 서랍 하나에는 동일한 개수의 검은 양말과 흰 양말이 들어있다. 다른 서랍에는 동일한 개수의 빨간 양말, 녹색 양말, 파랑 양말이 들어있다. 내가 임의로 서랍 하나를 고른 후, 역시 임의로 두 짝의 양말을 골랐는데 짝이 맞았다. 이 때 이 양말이 흰색일 확률은 얼마일까? 
- 문제를 간단히 하기 위해, 양 서랍에는 매우 많은 양말이 들어 있어 양말 한 짝을 꺼낸 후에도 확률 변화에 거의 영향을 미치지 않는다고 가정하자.

# 문제 3-4

- 앨비스 프레슬리에게 쌍둥이 형제가 있었다(태어날 때 사망). 앨비스 프레슬리가 일란성 쌍둥이일 가능성은 얼마인가?
- Hint: 1935년, 쌍둥이의 2/3 가량이 이란성이고 1/3이 일란성이었다.